# Function Calling

This is a simple example of how to enable agents to use functions.

In [7]:
# First, import the agent_callable decorator from botplayers.
from botplayers import agent_callable, Agent

In [8]:
# Then, define a function that can be called by agents.
# 1. The function must be decorated with agent_callable.
# 2. The function can have any number of arguments.
# 3. The function can return any JSON-serializable object.
# 4. [IMPORTANT] Tell agents how to call this function by writing a docstring.
@agent_callable()
def calculator(python_math_expression: str):
    """
    A simple calculator that can do basic math operations.
    Use math module to call functions or use constants like 
        math.sqrt, math.sin, math.cos, math.tan, math.asin, math.acos, math.atan, 
        math.log, math.log10, math.exp, math.e, math.pi.

    Args:
        python_math_expression: a python math expression.
    """
    import math
    return {'result': eval(python_math_expression, {'math': math})}

In [9]:
# Now, let's see how to call this function from agents.
# First, create an agent. Use prompt to tell agents what they can do.
agent = Agent(
    name='Bot', 
    prompt="You are a helpful bot. You can use functions to accomplish tasks.",
    interactive_objects=[calculator],
    function_call_repeats=15, 
    ignore_none_function_messages=False)

In [10]:
# Then, run the agent.
user_message = "Could please tell me what is 234*(212-3456)/(121-21241)?"
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act()

Bot received a message: Could please tell me what is 234*(212-3456)/(121-21241)?
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '234*(212-3456)/(121-21241)'}
        response: {'result': 35.94204545454546}
Bot >> 
The result of the expression 234*(212-3456)/(121-21241) is approximately 35.94.


# World Interaction

Agents can also use functions to interact with a stateful environment, e.g the world.

In [11]:
# Now, lets import the InteractiveSpace class from the botplayers package
from botplayers import InteractiveSpace

# Define simple environment with a simple state.
class Env(InteractiveSpace):
    messages: dict[str, list] = dict()

    # Define a callable classmethod that can be called by agents.
    # Note that, the `self` and the `agent` parameters are invisible to agents.
    # The `self` parameter is automatically filled by the environment object.
    # The `agent` parameter is automatically filled by the agent object.
    # Therefore, do not mention `self` and `agent` in the docstring.
    @agent_callable()
    def store(self, agent: Agent, message: str):
        """
        Store a message from an agent.

        Args:
            message: the message to store.
        """
        if agent.name not in self.messages:
            self.messages[agent.name] = []
        self.messages[agent.name].append(message)
        return 'done'

    @agent_callable()
    def get_stored_messages(self, agent: Agent):
        """
        Get all stored messages.

        Returns:
            messages: the list of messages.
        """
        return {'messages': self.messages.get(agent.name, [])}

In [12]:
# Now lets create an instance of Env.
env = Env()

# Let the agent interact with the environment.
agent.add_interactive_object(env)

# Note that botplayers supports error feedback, so that agents can learn from their mistakes.
user_message = 'Please propose three complex math calculation problems. First compute them, then store both the questions and answers.'
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act()

user_message = 'Please show me the questions and answers.'
agent.receive_message({'role': 'user', 'content': user_message})
agent.think_and_act()

Bot received a message: Please propose three complex math calculation problems. First compute them, then store both the questions and answers.
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '7^3 + (sqrt(25) * cos(pi/4)) - log10(1000)'}
        error: name 'sqrt' is not defined
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '7**3 + (sqrt(25) * cos(pi/4)) - log10(1000)'}
        error: name 'sqrt' is not defined
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '7**3 + ((25**0.5) * cos(math.pi/4)) - math.log10(1000)'}
        error: name 'cos' is not defined
Bot >> 
    Bot is calling function calculator ...
        with arguments {'python_math_expression': '7**3 + ((25**0.5) * math.cos(math.pi/4)) - math.log10(1000)'}
        response: {'result': 343.53553390593277}
Bot >> 
    Bot is calling function store ...
        with argume